In [1]:
%run Reddit_import_and_functions.ipynb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429116 entries, 0 to 429115
Data columns (total 9 columns):
Unnamed: 0      429116 non-null int64
Unnamed: 0.1    429116 non-null object
date            429115 non-null object
full_link       429115 non-null object
subreddit       429115 non-null object
url             429115 non-null object
title           429115 non-null object
author          429115 non-null object
selftext        177171 non-null object
dtypes: int64(1), object(8)
memory usage: 29.5+ MB


In [6]:
subreddit1 = 'Environment'
subreddit2 = 'Cryptocurrency'
headers = {'User-agent' : 'General Assembly Project 3-J'}
#zerotime = int(datetime.datetime.utcnow().timestamp())
zerotime = 1545098409 # will grab by each previous hour
params = {'size':500,'subreddit':subreddit1,'after':(zerotime-60*60*24*3),'before':zerotime}
# Pulling posts starting from 2 years ago onwards, 2 days per pull to stay under 500 post limit

url = 'https://api.pushshift.io/reddit/search/submission/'
response = requests.get(url, headers=headers, params=params)
print('Status Code: ',response.status_code)  # testing if connection is accepting my IP

Status Code:  200


In [114]:
features = ['date','permalink','subreddit','url','title','author','selftext'] 
# date is only column not directly in json

df = pd.DataFrame(columns = features)

In [123]:
def jsontocsv(json, csvfile) : # efficient, fastest & stores data at each step (better than jsontodf())
                               # will re-index based on post id when re-create dataframe
    df = pd.DataFrame(columns = features)
    for post_dict in json['data'] :
        
        date_tuple = datetime.datetime.utcfromtimestamp(post_dict['created_utc'])
        year = date_tuple.year
        month = date_tuple.month
        day = date_tuple.day
        hour = date_tuple.hour
        second = date_tuple.second
        try : post_dict['selftext']
        except KeyError : post_dict['selftext'] = ''
        
        df_dict = {feature : post_dict[feature] for feature in features[1:]}
        df_dict['date'] = (year, month, day, hour, second)
        df_dict['id'] = post_dict['id']
        df_dict['url'] = df_dict['url'].replace(',','').replace(';','')
        
        df = df.append(df_dict, ignore_index=True)
        
    df.to_csv(csvfile, mode='a', header=False)
    return df

In [5]:
# Older version of json converting function, csv version is much faster
def jsontodf(json, df) :
    for post_dict in json['data'] :
        
        date_tuple = datetime.datetime.utcfromtimestamp(post_dict['retrieved_on'])
        year = date_tuple.year
        month = date_tuple.month
        day = date_tuple.day
        hour = date_tuple.hour
        second = date_tuple.second
        post_dict['date'] = (year, month, day, hour, second)
        
        for feature in features :  # dataframe indexed by reddit's post id
            try :                  # this ensures no duplicates, as inputting same post again will just
                                   # overwrite exact oldy copy
                df.loc[post_dict['id'], feature] = post_dict[feature] 
            except KeyError :
                df.loc[post_dict['id'], feature] = ""

In [131]:
def addposts(url, params, df=pd.DataFrame(), csvfile=None, headers={}) : # adds all posts to df unless over 500 in one pull
    #numrows3 = 0
    for i in range(1*122) :  # 8 days per call, max 96x500=48,000 posts pulled
        res = requests.get(url, params=params, headers=headers)
        if res.status_code == 200 :
            #numrows, _ = df.shape
            #jsontodf(res.json(), df)
            df = jsontocsv(res.json(), csvfile)
            #numrows2, _ = df.shape
            if (i % 1) == 0 :
                #print(numrows2-numrows, numrows-numrows3, 'rows to df', datetime.datetime.utcfromtimestamp(params['before']))
                print(df.shape[0], 'rows to df', datetime.datetime.utcfromtimestamp(params['before']))
            #numrows3 = numrows
            params['after'] = params['after']-60*60*24*3
            params['before'] = params['before']-60*60*24*3
        else :
            print(res.status_code)
            break
        time.sleep(0.1)
    return params

In [11]:
response.json()['data'][10]

{'author': 'neerjsn',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_2oen7oyu',
 'author_patreon_flair': False,
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1545061541,
 'domain': 'i.redd.it',
 'full_link': 'https://www.reddit.com/r/environment/comments/a70w7w/top_10_thing_you_might_not_know_about_pollution/',
 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0},
 'id': 'a70w7w',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': True,
 'is_robot_indexable': False,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 2,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'all_ads',
 'permalink': '/r/environment/comment

In [91]:
df = pd.DataFrame(columns = features)
df1 = pd.DataFrame(columns = features)
df.shape

(0, 7)

In [117]:
csvfile = 'environmentFinal.csv'

In [134]:
newparams = {'size': 500,
 'subreddit': 'Environment',
 'after': 1324000809,
 'before': 1324260009}
# For environment subreddit only: after variable is 24*3 hours interval from before variable

In [126]:
params

{'size': 500,
 'subreddit': 'Environment',
 'after': 1544839209,
 'before': 1545098409}

In [135]:
addposts(url, newparams, csvfile=csvfile, df=df, headers=headers)

156 rows to df 2011-12-19 02:00:09
224 rows to df 2011-12-16 02:00:09
160 rows to df 2011-12-13 02:00:09
227 rows to df 2011-12-10 02:00:09
226 rows to df 2011-12-07 02:00:09
169 rows to df 2011-12-04 02:00:09
223 rows to df 2011-12-01 02:00:09
139 rows to df 2011-11-28 02:00:09
171 rows to df 2011-11-25 02:00:09
161 rows to df 2011-11-22 02:00:09
211 rows to df 2011-11-19 02:00:09
175 rows to df 2011-11-16 02:00:09
208 rows to df 2011-11-13 02:00:09
233 rows to df 2011-11-10 02:00:09
153 rows to df 2011-11-07 02:00:09
191 rows to df 2011-11-04 02:00:09
156 rows to df 2011-11-01 02:00:09
221 rows to df 2011-10-29 02:00:09
181 rows to df 2011-10-26 02:00:09
175 rows to df 2011-10-23 02:00:09
206 rows to df 2011-10-20 02:00:09
126 rows to df 2011-10-17 02:00:09
179 rows to df 2011-10-14 02:00:09
94 rows to df 2011-10-11 02:00:09
169 rows to df 2011-10-08 02:00:09
139 rows to df 2011-10-05 02:00:09
156 rows to df 2011-10-02 02:00:09
207 rows to df 2011-09-29 02:00:09
129 rows to df 2011-0

{'size': 500,
 'subreddit': 'Environment',
 'after': 1292378409,
 'before': 1292637609}

In [85]:
newparams = {'size': 500,
 'subreddit': 'environment',
 'after': 1482180900,
 'before': 1482353700}

In [100]:
1513562409-60*60*24*3

1513303209

In [83]:
(1482180900-1481576100)/(3600*24)

7.0

In [33]:
datetime.datetime.utcfromtimestamp(1482008100)

datetime.datetime(2016, 12, 17, 20, 55)

In [45]:
# file = 'cryptocurrency12_26_17to9_18_17.csv'
# df.to_csv(file, mode='a', header=False)
# df1.to_csv(file, mode='a', header=False)